In [95]:
import operator
from typing import Annotated, TypedDict, List
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.pregel import Pregel


In [96]:
# --- 1. Define State (No operator.add) ---
class State(TypedDict):
    messages: List[str]

# --- 2. Define Nodes ---

def node_1(state: State):
    print("Executing node_1")
    # Initialize the list
    return {"messages": ["Hello from node 1 -> "]}

def sub_node_1(state: State):
    print("Executing sub_node_1")
    # Manual Append: Get existing -> Add new -> Return full list
    current_list = state.get("messages", [])
    new_list = current_list + [" ... (sub_node_1) Interrupting! -> "]
    return {"messages": new_list}

def sub_node_2(state: State):
    print("Executing sub_node_2")
    # Manual Append
    current_list = state.get("messages", [])
    new_list = current_list + [" ... (sub_node_2) Resumed! -> "]
    return {"messages": new_list}

def node_3(state: State):
    print("Executing node_3")
    # Manual Append
    current_list = state.get("messages", [])
    new_list = current_list + ["Hello from node 3!"]
    return {"messages": new_list}

In [98]:
# --- 3. Build Subgraph ---

sub_builder = StateGraph(State)
sub_builder.add_node("sub_node_1", sub_node_1)
sub_builder.add_node("sub_node_2", sub_node_2)

sub_builder.add_edge(START, "sub_node_1")
sub_builder.add_edge("sub_node_1", "sub_node_2")
sub_builder.add_edge("sub_node_2", END)

# Compile Subgraph with Interrupt
subgraph = sub_builder.compile(interrupt_before=["sub_node_2"])

# --- 4. Build Parent Graph ---

parent_builder = StateGraph(State)
parent_builder.add_node("node_1", node_1)
parent_builder.add_node("subgraph_wrapper", subgraph)
parent_builder.add_node("node_3", node_3)

parent_builder.add_edge(START, "node_1")
parent_builder.add_edge("node_1", "subgraph_wrapper")
parent_builder.add_edge("subgraph_wrapper", "node_3")
parent_builder.add_edge("node_3", END)

checkpointer = MemorySaver()
graph = parent_builder.compile(checkpointer=checkpointer)

In [ ]:
def resume_hybrid_graph_with_new_message(graph: Pregel, new_msg: str, config: dict, subgraph_map: dict):

    # Helper function to find the first parent node of a given target node in a graph.
    def get_first_parent(graph_runnable, target_node: str) -> str:
        """
        Returns the name of the first node that has an edge pointing TO the target_node.
        Returns '__start__' if the node is the first one in the graph.
        """
        graph_def = graph_runnable.get_graph()
        
        # Check edges to find who points to 'target_node'
        for edge in graph_def.edges:
            if edge.target == target_node:
                # We found the parent!
                return edge.source
                
        return None

    print(f"\n[Auto-Resumer] Inspecting graph for thread: {config['configurable']['thread_id']}")
    
    # 1. Fetch State with Subgraphs (Crucial to see inside)
    state_snapshot = graph.get_state(config, subgraphs=True)
    
    if not state_snapshot.tasks:
        print("[Auto-Resumer] No active tasks found. Is the graph paused?")
        return
    
    # 2. Identify the Active Subgraph Task
    # In a paused state, tasks[0] usually represents the subgraph wrapper node
    subgraph_task = state_snapshot.tasks[0]
    subgraph_node_name_in_parent = subgraph_task.name
    
    # 3. Get the Subgraph's Internal State
    subgraph_snapshot = subgraph_task.state
    if not subgraph_snapshot:
        print("[Auto-Resumer] Task found, but it's not a subgraph (no internal state).")
        return

    # 4. Identify the Halted Node inside the Subgraph
    # 'next' contains the tuple of nodes scheduled to run (e.g., 'sub_node_2')
    if not subgraph_snapshot.next:
        print("[Auto-Resumer] Subgraph is active but has no next steps?")
        return
        
    halted_node = subgraph_snapshot.next[0]
    print(f"[Auto-Resumer] Halted at node: '{halted_node}' (inside '{subgraph_node_name_in_parent}')")

    # 5. Fetch the Compiled Subgraph Object
    subgraph_runnable = subgraph_map[subgraph_node_name_in_parent]
    
    # 6. Find the Parent of the Halted Node
    # We need to know who runs BEFORE the halted node to attribute the update correctly.
    parent_node = get_first_parent(subgraph_runnable, halted_node)
    
    if not parent_node:
        print(f"[Auto-Resumer] Could not determine parent for '{halted_node}'. Defaulting to halted node itself.")
        parent_node = halted_node # Fallback
    else:
        print(f"[Auto-Resumer] Determined parent node is: '{parent_node}'")

    # 7. Prepare the Update (Manual Append)
    current_messages = subgraph_snapshot.values["messages"]
    updated_messages = current_messages + [new_msg]
    
    # 8. Update State
    # We use the SUBGRAPH'S config and the CALCULATED parent node
    print(f"[Auto-Resumer] Injecting message via '{parent_node}'...")
    graph.update_state(
        subgraph_snapshot.config,
        {"messages": updated_messages},
        as_node=parent_node
    )
    
    # 9. Resume Execution
    print("[Auto-Resumer] Resuming execution...")
    # We stream/invoke with None to unpause
    for event in graph.stream(None, config):
        pass
        
    print("[Auto-Resumer] Done.")

In [104]:
# ... (Assume 'graph' is compiled and has run up to the interrupt) ...

# Define your config
config = {"configurable": {"thread_id": "auto_resume_demo"}}

# 1. Run initially to hit the interrupt
print("--- Initial Run ---")
for event in graph.stream({"messages": []}, config):
    pass

# 2. Call the magic function
# This single line handles the lookup, the parent finding, the update, and the resume.
resume_hybrid_graph_with_new_message(
    graph, 
    "hi from human (auto-injected) -> ", 
    config,
    {
        "subgraph_wrapper": subgraph
    }
)

# 3. Verify Final State
final = graph.get_state(config)
print("\n--- FINAL OUTPUT ---")
print("".join(final.values["messages"]))

--- Initial Run ---
Executing node_1
Executing sub_node_1

[Auto-Resumer] Inspecting graph for thread: auto_resume_demo
[Auto-Resumer] Halted at node: 'sub_node_2' (inside 'subgraph_wrapper')
[Auto-Resumer] Determined parent node is: 'sub_node_1'
[Auto-Resumer] Injecting message via 'sub_node_1'...
[Auto-Resumer] Resuming execution...
Executing sub_node_2
Executing node_3
[Auto-Resumer] Done.

--- FINAL OUTPUT ---
Hello from node 1 ->  ... (sub_node_1) Interrupting! -> hi from human (auto-injected) ->  ... (sub_node_2) Resumed! -> Hello from node 3!
